In [ ]:
import random
import numpy as np

# Objective function
def objective_function(params):
    a1, a2, a3, a4, a5 = params
    return 1 + 2*a1 + (3*a2 - 1) + 3*a3 + 2*(a4**2) + (5*a5 + 2)

In [ ]:
# Initialize PSO parameters
n_particles = 20
w_max = 0.9
w_min = 0.3
c1 = 2.0
c2 = 2.0
max_iterations = 100

In [ ]:
# Define the search space
bounds = [(10, 60), (15, 30), (25, 75), (10, 30), (10, 50)]

In [ ]:
# Initialize particles
particles = []
velocities = []
best_positions = []
best_scores = []

In [ ]:
for _ in range(n_particles):
    particle = [random.uniform(bound[0], bound[1]) for bound in bounds]
    velocity = [random.uniform(-1, 1) for _ in range(len(bounds))]
    particles.append(particle)
    velocities.append(velocity)
    best_positions.append(particle)
    best_scores.append(objective_function(particle))

In [ ]:
# Initialize global best position and score
global_best_position = particles[np.argmax(best_scores)]
global_best_score = max(best_scores)

In [ ]:
# Run PSO iterations
for iteration in range(max_iterations):
    # Update inertia weight
    w = w_max - (w_max - w_min) * (iteration / max_iterations)

    for i in range(n_particles):
        # Update velocities
        r1 = random.random()
        r2 = random.random()
        velocities[i] = (
            w * np.array(velocities[i]) +
            c1 * r1 * (np.array(best_positions[i]) - np.array(particles[i])) +
            c2 * r2 * (np.array(global_best_position) - np.array(particles[i]))
        )

        # Update positions
        particles[i] = np.array(particles[i]) + velocities[i]

        # Apply bounds
        particles[i] = [max(min(x, bound[1]), bound[0]) for x, bound in zip(particles[i], bounds)]

        # Update personal best
        score = objective_function(particles[i])
        if score > best_scores[i]:
            best_positions[i] = particles[i]
            best_scores[i] = score

    # Update global best
    if max(best_scores) > global_best_score:
        global_best_score = max(best_scores)
        global_best_position = best_positions[best_scores.index(global_best_score)]

In [ ]:
# Print the optimal solution
print("Optimal solution:")
print("a1 =", global_best_position[0])
print("a2 =", global_best_position[1])
print("a3 =", global_best_position[2])
print("a4 =", global_best_position[3])
print("a5 =", global_best_position[4])
print("Objective function value =", global_best_score)

Optimal solution:
a1 = 60
a2 = 30
a3 = 75
a4 = 30
a5 = 50
Objective function value = 2487


Another Approach

In [ ]:
import numpy as np
import random

In [ ]:
def objective_function(x):
    """Calculates the value of the objective function"""
    a1, a2, a3, a4, a5 = x
    value = 1 + 2*a1 + (3*a2-1) + 3*a3 + 2*(a4**2) + (5*a5+2)
    return value

In [ ]:
class Particle:
    def __init__(self, dimension, bounds):
        """
        Initializes a particle
        :param dimension: Number of dimensions of the problem
        :param bounds: List of tuples, each tuple representing bounds for a dimension [(lower, upper)]
        """
        self.dimension = dimension
        self.bounds = bounds

        self.position = np.array([random.uniform(*bounds[i]) for i in range(dimension)])
        self.velocity = np.array([0.0 for _ in range(dimension)])

        self.pbest_position = self.position.copy()
        self.pbest_value = float('-inf')  # For maximization

    def update_velocity(self, gbest_position, w, c1, c2):
        """
        Updates particle velocity based on PSO update rule
        :param gbest_position: Global best position
        :param w: Inertia weight
        :param c1: Cognitive coefficient
        :param c2: Social coefficient
        """
        r1 = random.random()
        r2 = random.random()

        cognitive = c1 * r1 * (self.pbest_position - self.position)
        social = c2 * r2 * (gbest_position - self.position)
        self.velocity = w * self.velocity + cognitive + social

    def update_position(self):
        """Updates particle position based on velocity"""
        self.position += self.velocity

        # Enforce bounds
        for i in range(self.dimension):
            self.position[i] = np.clip(self.position[i], self.bounds[i][0], self.bounds[i][1])

    def evaluate(self):
        """Calculates the particle's fitness and updates personal best (pbest)"""
        self.fitness = objective_function(self.position)
        if self.fitness > self.pbest_value:
            self.pbest_value = self.fitness
            self.pbest_position = self.position.copy()

In [ ]:
def PSO(objective_function, bounds, n_particles, iterations, w_max, w_min, c1, c2):
    """ Implements PSO algorithm """

    dimension = len(bounds)
    particles = [Particle(dimension, bounds) for _ in range(n_particles)]

    gbest_value = float('-inf')  # For maximization
    gbest_position = np.zeros(dimension)

    for _ in range(iterations):
        # Update global best
        for particle in particles:
            particle.evaluate()
            if particle.pbest_value > gbest_value:
                gbest_value = particle.pbest_value
                gbest_position = particle.pbest_position.copy()

        # Update particles
        w = w_max - (_ / iterations) * (w_max - w_min)  # Linear inertia update
        for particle in particles:
            particle.update_velocity(gbest_position, w, c1, c2)
            particle.update_position()

    return gbest_position, gbest_value

In [ ]:
# Parameters
n_particles = 20
iterations = 100
w_max = 0.9
w_min = 0.3
c1 = 2
c2 = 2

bounds = [(10, 60), (15, 30), (25, 75), (10, 30), (10, 50)]

# Solve the optimization problem
best_position, best_value = PSO(objective_function, bounds, n_particles, iterations, w_max, w_min, c1, c2)

print("Optimal Solution:", best_position)
print("Maximum Value:", best_value)

Optimal Solution: [60. 30. 75. 30. 50.]
Maximum Value: 2487.0
